# Overview

This notebook demonstrates how to use the `sammy.paramter` module to load, construct and manipulate the parameter file (`*.par`) of the SAMMY suite.

In [ ]:
import pathlib
import tempfile

from pleiades.sammy.parfile import SammyParameterFile

## Construct a parameter object manually

Let's build a simple parameter file from scratch.

In [2]:
from pleiades.sammy.parameters.helper import VaryFlag
from pleiades.sammy.parameters.resonance import ResonanceCard, ResonanceEntry

resonance_0 = ResonanceEntry(
    resonance_energy=0.0253,
    capture_width=0.1,
    channel1_width=0.2,
    channel2_width=0.3,
    channel3_width=0.4,
    vary_energy=VaryFlag.NO,
    vary_capture_width=VaryFlag.NO,
    vary_channel1_width=VaryFlag.YES,
    vary_channel2_width=VaryFlag.NO,
    vary_channel3_width=VaryFlag.YES,
    igroup=1,
)

resonance_1 = ResonanceEntry(
    resonance_energy=0.0253,
    capture_width=0.1,
    channel1_width=0.2,
    channel2_width=0.3,
    channel3_width=0.4,
    vary_energy=VaryFlag.NO,
    vary_capture_width=VaryFlag.NO,
    vary_channel1_width=VaryFlag.NO,
    vary_channel2_width=VaryFlag.YES,
    vary_channel3_width=VaryFlag.NO,
    igroup=2,
)

resonance_card = ResonanceCard(entries=[resonance_0, resonance_1])

print(resonance_card)

entries=[ResonanceEntry(resonance_energy=0.0253, capture_width=0.1, channel1_width=0.2, channel2_width=0.3, channel3_width=0.4, vary_energy=<VaryFlag.NO: 0>, vary_capture_width=<VaryFlag.NO: 0>, vary_channel1=<VaryFlag.NO: 0>, vary_channel2=<VaryFlag.NO: 0>, vary_channel3=<VaryFlag.NO: 0>, igroup=1), ResonanceEntry(resonance_energy=0.0253, capture_width=0.1, channel1_width=0.2, channel2_width=0.3, channel3_width=0.4, vary_energy=<VaryFlag.NO: 0>, vary_capture_width=<VaryFlag.NO: 0>, vary_channel1=<VaryFlag.NO: 0>, vary_channel2=<VaryFlag.NO: 0>, vary_channel3=<VaryFlag.NO: 0>, igroup=2)]


In [3]:
from pleiades.sammy.parameters.radius import RadiusCard, RadiusParameters

radius_parameters = RadiusParameters(
    effective_radius=1.0,
    true_radius=2.0,
    channel_mode=1,
    vary_effective=VaryFlag.YES,
    vary_true=VaryFlag.NO,
    spin_groups=[1, 2, 3],
    channels=[1, 2, 3],
)

radius_card = RadiusCard(parameters=radius_parameters)

print(radius_card)

parameters=RadiusParameters(effective_radius=1.0, true_radius=2.0, channel_mode=1, vary_effective=<VaryFlag.YES: 1>, vary_true=<VaryFlag.NO: 0>, spin_groups=[1, 2, 3], channels=[1, 2, 3]) particle_pair=None orbital_momentum=None relative_uncertainty=None absolute_uncertainty=None


now let's create a parameter object to hold everything

In [4]:
par_file = SammyParameterFile(
    fudge=0.2,
    resonance=resonance_card,
    radius=radius_card,
)

print(par_file)

fudge=0.2 resonance=ResonanceCard(entries=[ResonanceEntry(resonance_energy=0.0253, capture_width=0.1, channel1_width=0.2, channel2_width=0.3, channel3_width=0.4, vary_energy=<VaryFlag.NO: 0>, vary_capture_width=<VaryFlag.NO: 0>, vary_channel1=<VaryFlag.NO: 0>, vary_channel2=<VaryFlag.NO: 0>, vary_channel3=<VaryFlag.NO: 0>, igroup=1), ResonanceEntry(resonance_energy=0.0253, capture_width=0.1, channel1_width=0.2, channel2_width=0.3, channel3_width=0.4, vary_energy=<VaryFlag.NO: 0>, vary_capture_width=<VaryFlag.NO: 0>, vary_channel1=<VaryFlag.NO: 0>, vary_channel2=<VaryFlag.NO: 0>, vary_channel3=<VaryFlag.NO: 0>, igroup=2)]) external_r=None broadening=None unused_correlated=None normalization=None radius=RadiusCard(parameters=RadiusParameters(effective_radius=1.0, true_radius=2.0, channel_mode=1, vary_effective=<VaryFlag.YES: 1>, vary_true=<VaryFlag.NO: 0>, spin_groups=[1, 2, 3], channels=[1, 2, 3]), particle_pair=None, orbital_momentum=None, relative_uncertainty=None, absolute_uncertaint

we can also view the string format before writing to a file

In [5]:
print(par_file.to_string())

 2.5300E-02 1.0000E-01 2.0000E-01 3.0000E-01 4.0000E-01 0 0 0 0 0 1
 2.5300E-02 1.0000E-01 2.0000E-01 3.0000E-01 4.0000E-01 0 0 0 0 0 2

0.2000    
RADII are in KEY-WORD format
Radius= 1.0 2.0
Flags= 1 0
Group= 1 Channels= 1 2 3
Group= 2 Channels= 1 2 3
Group= 3 Channels= 1 2 3



then we can write the parameter object to a file

In [6]:
with tempfile.TemporaryDirectory() as tmpdir:
    par_file_path = pathlib.Path(tmpdir) / "sammy.par"
    par_file.to_file(par_file_path)

    with open(par_file_path, "r") as f:
        print(f.read())

 2.5300E-02 1.0000E-01 2.0000E-01 3.0000E-01 4.0000E-01 0 0 0 0 0 1
 2.5300E-02 1.0000E-01 2.0000E-01 3.0000E-01 4.0000E-01 0 0 0 0 0 2

0.2000    
RADII are in KEY-WORD format
Radius= 1.0 2.0
Flags= 1 0
Group= 1 Channels= 1 2 3
Group= 2 Channels= 1 2 3
Group= 3 Channels= 1 2 3



## Load from string/file

We can also directly load a parameter file from a string or a file.

In [7]:
with tempfile.TemporaryDirectory() as tmpdir:
    par_file_path = pathlib.Path(tmpdir) / "sammy.par"
    par_file.to_file(par_file_path)

    par_file_read = SammyParameterFile.from_file(par_file_path)
    print(par_file_read)
    print(par_file_read.to_string())

fudge=0.2 resonance=ResonanceCard(entries=[ResonanceEntry(resonance_energy=0.0253, capture_width=0.1, channel1_width=0.2, channel2_width=0.3, channel3_width=0.4, vary_energy=<VaryFlag.NO: 0>, vary_capture_width=<VaryFlag.NO: 0>, vary_channel1=<VaryFlag.NO: 0>, vary_channel2=<VaryFlag.NO: 0>, vary_channel3=<VaryFlag.NO: 0>, igroup=1), ResonanceEntry(resonance_energy=0.0253, capture_width=0.1, channel1_width=0.2, channel2_width=0.3, channel3_width=0.4, vary_energy=<VaryFlag.NO: 0>, vary_capture_width=<VaryFlag.NO: 0>, vary_channel1=<VaryFlag.NO: 0>, vary_channel2=<VaryFlag.NO: 0>, vary_channel3=<VaryFlag.NO: 0>, igroup=2)]) external_r=None broadening=None unused_correlated=None normalization=None radius=RadiusCard(parameters=RadiusParameters(effective_radius=1.0, true_radius=2.0, channel_mode=1, vary_effective=<VaryFlag.YES: 1>, vary_true=<VaryFlag.NO: 0>, spin_groups=[3], channels=[1, 2, 3]), particle_pair=None, orbital_momentum=None, relative_uncertainty=None, absolute_uncertainty=None